In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
# Constants
TOTAL_FIELDS = 259
DATADIR = '../data/'
RESULTDIR = '../results/'
FILE = 'uniswap.csv'

In [4]:
# Functions
def get_gas_table():
    gas_price_data = read_csv(DATADIR + 'gas_table.csv')
    gas_table = {}
    for fields in gas_price_data:
        if len(fields) == 4:
            gas_table[int(fields[0], 16)] = {
                'current_gas_cost': fields[1],
                'proposed_gas_cost': fields[2],
                'mnemonic': fields[3],
            }
    return gas_table

# Calculates new gas used based on the gas_table
def apply_gas_table(line):
    gas_table = get_gas_table()
    for ix, op in enumerate(line):
        print(ix, op)

In [5]:
def read_csv(file):
    fin = open(file, 'r')
    data = fin.read()
    fin.close()
    data = data.split('\n')
    data = [line.split(',') for line  in data]
    return data

In [6]:
def convert_data(data):
    final_data = []
    for line in data:
        new_line = [int(elem) if i > 1 else elem for i, elem in enumerate(line)]
        final_data.append(new_line)
    return final_data

def write_csv(fname, data):
    fout = open(fname, 'w')
    fout.write(data)
    fout.close()    


def convert_to_csv(data):
    out_str = ''
    for line in data:
        line = [x if isinstance(x, str) else str(x) for x in line]
        out_str += ','.join(line) + '\n'
    return out_str
        
def apply_gas_table(columns_name, gas_table, line):
    if len(line) != TOTAL_FIELDS:
        return None
    
    tx_gas_used = int(line[2])
    
    if tx_gas_used == 0:
        print('Ignoring transaction', line[1], 'gasUsed:', tx_gas_used)
        return None
    
    opdata = line[3:]
    
    tx_total = 0
    tx_new_total = 0
    line_out = []
    for ix, count in enumerate(opdata):
        opnum = ix
        opname = columns_name[opnum+3]
        if ix in gas_table.keys():
            cost = gas_table[opnum]['current_gas_cost']
            total = int(count) * int(cost)
            new_cost = gas_table[opnum]['proposed_gas_cost']
            new_total = int(count) * int(new_cost)
            
            tx_total += total
            tx_new_total += new_total
            
    diff = tx_total - tx_new_total
    # Substracts the difference to the tx gasUsage
    new_tx_gas_used = tx_gas_used - diff
    decrease = tx_gas_used - new_tx_gas_used
    decrease_perc = (decrease / float(tx_gas_used)) * 100
    
    #         [Block #, Tx, Gas Used] + [new_gas_used, difference, percentage] + [opdata]
    line_out = line[:2] + [int(line[2]), int(new_tx_gas_used), int(diff), float("{:.2f}".format(decrease_perc))] + line[3:]
    return line_out


In [7]:
def analyze_data(filename):
    csv = read_csv(filename)
    columns_name = csv[0]
    columns_value = csv[1]
    data = convert_data(csv[2:])
    
    data_result = []

    # Add new fields: Gas Used Rp. and Difference
    for line in data:
        new_line = apply_gas_table(columns_name, gas_table, line)
        if new_line:
            data_result.append(new_line)
            
    return columns_name, columns_value, data_result

In [8]:
def get_sample(data, columns, filter_columns):
    df = pd.DataFrame(data, columns=columns)
    return df[filter_columns]

In [9]:
gas_table = get_gas_table()

In [10]:
results = {}
for input_file in ['histogram.csv', 'uniswap.csv', '1inch.csv', 'usdc.csv', 'usdt.csv', 'aave.csv', 'mm_swap.csv']:
    columns_name, columns_value, data_result = analyze_data(DATADIR + input_file)
    data_result_csv = convert_to_csv(data_result)
    write_csv(RESULTDIR + input_file, data_result_csv)
    new_columns = columns_name[0:3] + ["Gas Used RP", "Difference", "%"] + columns_name[3:]
    results[input_file.split('.')[0]] = data_result

Ignoring transaction 0xc75a1a1ff3b671eaf1e02139573d448c665fe183aa39d678b616db9549c83588 gasUsed: 0
Ignoring transaction 0x7b8a5b9663a3be6015cb53224d42896cb2ca8cd7c82702200fdc304ce1b116b0 gasUsed: 0
Ignoring transaction 0xf9a29b31b92bae188dd3b4e526506d4094311a5b03ac7685497adf0b15a13289 gasUsed: 0
Ignoring transaction 0xb8af5e42a5cc84c71fe220792b2d3c0685898252a216ff9f7e4d20bff12538f7 gasUsed: 0
Ignoring transaction 0x8f2bcfcce8f05f690de078da1d9eb93ac007b7cc0b8677c6f9a9ec432389a87a gasUsed: 0
Ignoring transaction 0x860878d13ad812d647d9966d474e3581583f01f6f93f35bab85c14da0327a747 gasUsed: 0
Ignoring transaction 0x3a18703fb65864e753dfcf0bf0d3503871577ce6bfce370f55ac4c922c1a605b gasUsed: 0
Ignoring transaction 0x1c392fb0939bd3785b5876b0398efef1881d574cdeafa37b073afe7faac8b7ae gasUsed: 0
Ignoring transaction 0x08e91987a701c5005ae9118a6572ce66a0d0cc1f4d7ebb228038fa0adc6eb7d7 gasUsed: 0
Ignoring transaction 0x29fb33ae2c2360eb3b0eff87342be17dfe8095104e690d808c1864aa685b434d gasUsed: 0
Ignoring t

Ignoring transaction 0xc984ec00f69f7376e2b27f3c32a5936d439d46217b9fa1d0857bd06ec7aee7a3 gasUsed: 0
Ignoring transaction 0x45bbe059af21ce9c1783303e0dacfcbb9c73910a4de67f6c49cb48dda200a8cd gasUsed: 0
Ignoring transaction 0xb9cc3ccd5fdf01bb6823df62b683669c83f020e3e4a3c63871c3e242d0dff25c gasUsed: 0
Ignoring transaction 0xd911ba96f5846ac1e66dba0624659a6bc52ba0fd9abf378ea15ad866e575eaaa gasUsed: 0
Ignoring transaction 0x4ad7827b9c7b610c4275140efa19326e0494272571a9c9265927872b95c5cd85 gasUsed: 0
Ignoring transaction 0xabd5fe6c1cc4e0bddaf119277960e75fcc19164d3dc1fb3a852537c272d5c813 gasUsed: 0
Ignoring transaction 0x03ee2acb858d385ce0bcbe5f40df997dbc0f29813e5c3ca6ede6c2ceebe1e116 gasUsed: 0
Ignoring transaction 0xf785ef39e75acd3ca627142d82149aca60965e583cb012870f75741c69ff9ac8 gasUsed: 0
Ignoring transaction 0xb2570338ffaede0b17dc73cef4331215fcd15a6a42e48255acce0a4cc7ce4319 gasUsed: 0
Ignoring transaction 0x72a62c143c78da0a3e91de6f4828a7a4e18cec3484ca0c6e293d8fef4f633cf6 gasUsed: 0
Ignoring t

Ignoring transaction 0x17574a28ca5420ae788f29084327fd3e6310e1426eed26f55cf371f5d281395c gasUsed: 0
Ignoring transaction 0xc6b6df10eba4282261a06c4b13387f4f6b1bf2a00156f61b0ee2573f404aea7b gasUsed: 0
Ignoring transaction 0x90cbbd02e21926ed4fd421540e6412a7d76608a899e10bb29b385b1d99ce0879 gasUsed: 0
Ignoring transaction 0xbd196712d1a1618a49d05055f90425064e429d125d950b245b5eb30bd0e04704 gasUsed: 0
Ignoring transaction 0x2915eeb0226f83191e5e1334f7c043480a1b758773a2eed03d9e49eaab3f0112 gasUsed: 0
Ignoring transaction 0x79ceb90d04c6c776173f0fab289ee8e22ef923a79e5982f7a4d5bd2b00ef928a gasUsed: 0
Ignoring transaction 0x08dace9a563833db6570cfd47cc8dfc6555f16e63b86ecc5cfc284cef987e854 gasUsed: 0
Ignoring transaction 0x5e0a3020b91c2ac5762f3f0d39d0faf17bfe76dbd9c3c9b604ff7e6b2b311571 gasUsed: 0
Ignoring transaction 0xa2735cdc87947cb1aff708fe744f7f5819e22d357849abd5d51cc84e76d90b46 gasUsed: 0
Ignoring transaction 0xa98a0f94362e8f4051ee82638a610d6e9dd2bb1269416374299f0a4c4bd25750 gasUsed: 0
Ignoring t

Ignoring transaction 0x2d37fa110e3ffc692064e227fc6f76f9e6964f8d43e95479df6e857cd96781ac gasUsed: 0
Ignoring transaction 0x037869bd85366be87ef4271d65dda70a1eddedbec3bca0b0962ca132fecee3e1 gasUsed: 0
Ignoring transaction 0x3f54279c97bc66963c0c52ff2220e894dcbadcd7ed24e7116b83529d0bd2b3db gasUsed: 0
Ignoring transaction 0x9548fa8ae08acec8080018d0abf4bb0b265e4cf4fa5369da3c2e57497af86820 gasUsed: 0
Ignoring transaction 0xf5892fe8436b673631f698dec0571bfbc5201df259f9262d05f7f409ffa6a540 gasUsed: 0
Ignoring transaction 0x38aefd0b159a5cabc2818546561e2ae143da2f5d7f823e08071905d06da5f167 gasUsed: 0
Ignoring transaction 0x3141923914868ba39f5d5904de855859f0aa41e3a7f2376bf036c0d0531ac568 gasUsed: 0
Ignoring transaction 0x7efaeb5467f5190cf5eb1a15ea3f119dd960c7ab5e0f617f928ad79ea0edc9a6 gasUsed: 0
Ignoring transaction 0x9dec4af7482a96442604b09c4fe3ad7c8b0442b7f625adce6a6392e123de639a gasUsed: 0
Ignoring transaction 0xb9bde3eeee8ffba2f6a9e89e04f01369f36af34148f322e22288958f05f92161 gasUsed: 0
Ignoring t

Ignoring transaction 0xf9a3460c51a6749ae55bb824f84abe8f375d1210d1727704b713317fde66b573 gasUsed: 0
Ignoring transaction 0x7007bd32ab6a8cea6bec774fbbee43fb8af8d9fb604b58fa15b6a5948a58b6c5 gasUsed: 0
Ignoring transaction 0x8e2b29c2008134b099cfd9ed36fb1ab905b89aa692bd63e88e37493da48f210f gasUsed: 0
Ignoring transaction 0x841f7730437f13da5a281aa22cc9f6d6ecf5b8e701ed5020e81701946c6e7e60 gasUsed: 0
Ignoring transaction 0x77a51559057040d25ba2c2afe83d7fc596c853cbcc6b05653d0787246c4e90ab gasUsed: 0
Ignoring transaction 0x082cf7fab76a856a974a5bf6383b1c2482e363a9fec59ee98ac1d216b84f6498 gasUsed: 0
Ignoring transaction 0xa43c113c9785266e0c46997bd30751d5f9cba5e981c3761dac7557eaa5ac7dd9 gasUsed: 0
Ignoring transaction 0xae84d7bb7bb2845389f1d45c31d43548415cfb4b6e4a769728a02487ba93b5f8 gasUsed: 0
Ignoring transaction 0xa18f5f8fdc87ca7d023f8ab808de24a34e060394b23a59ca4166d16b14aeea56 gasUsed: 0
Ignoring transaction 0x5c8ede950e759d202e1f8b87b0c5646a5d096b43ce619fcec34f911076f3a323 gasUsed: 0
Ignoring t

In [11]:
filter_columns = ['Block #', 'Transactions', 'Gas Used', 'Gas Used RP', 'Difference', '%']

# ALL Traced Contracts

In [12]:
contract = 'histogram' # ALL
print('Total traced transactions:', len(results[contract]))

Total traced transactions: 431515


In [13]:
sample = get_sample(results[contract], new_columns, filter_columns)
sample.head()

,Block #,Transactions,Gas Used,Gas Used RP,Difference,%
0,0xbbcd41,0xbeabf6680e1f868c6fa0979ab25a7524b19bdca7d97c...,13074,12595,479,3.66
1,0xbbcd41,0x5e857390c4f6ab1d157cc40a5d43bc3bcbc863a09e4f...,39267,38728,539,1.37
2,0xbbcd41,0x7cc8e7a4666fc6b269a6a4a4d56becbbd63b405247a0...,32706,32209,497,1.52
3,0xbbcd41,0x6af3205f62e69f7dc6993363bab674e87a1e18a694f0...,12925,12496,429,3.32
4,0xbbcd41,0x45a88b3daa39922cda59be85e91ef2223b603bb64de5...,44679,43923,756,1.69


## Average gasUsed, gasUsedRP, Difference

In [14]:
sample.iloc[:,2:].apply(np.mean)

Gas Used       78500.083344
Gas Used RP    74686.642063
Difference      3813.441280
%                  4.700983
dtype: float64

## Minimum gasUsed, gasUsedRP, Difference

In [15]:
sample.iloc[:,2:].apply(np.min)

Gas Used       6.00
Gas Used RP    2.00
Difference     4.00
%              0.01
dtype: float64

## Maximum gasUsed, gasUsedRP, Difference

In [16]:
sample.iloc[:,2:].apply(np.max)

Gas Used       14874491.00
Gas Used RP    14864787.00
Difference       530095.00
%                    66.67
dtype: float64

# AAVE

In [17]:
contract = 'aave'
print('Total traced transactions:', len(results[contract]))

Total traced transactions: 2004


In [18]:
sample = get_sample(results[contract], new_columns, filter_columns)
sample.head()

,Block #,Transactions,Gas Used,Gas Used RP,Difference,%
0,0xbbf35f,0x4000f94fabd0ee7de180299e0cd39c12d1f12c2cf1da...,244001,227501,16500,6.76
1,0xbbf362,0xa75da7fb949426815a605536578e98c2d89799077b50...,341844,315066,26778,7.83
2,0xbbf363,0x29f577204c90ce6bfb0416a0cc06cab850d3a13e6909...,270043,244683,25360,9.39
3,0xbbf366,0x48522ba0493bd323a0aed85343554f200f87d6903645...,244001,227501,16500,6.76
4,0xbbf367,0x34bd042b501f70e2cae1f6e0cf931c865d26ee11a84a...,209791,193246,16545,7.89


## Average gasUsed, gasUsedRP, Difference

In [19]:
sample.iloc[:,2:].apply(np.mean)

Gas Used       294614.779441
Gas Used RP    272707.087325
Difference      21907.692116
%                   7.343418
dtype: float64

# Minimum gasUsed, gasUsedRP, Difference

In [20]:
sample.iloc[:,2:].apply(np.min)

Gas Used       5326.0
Gas Used RP    4887.0
Difference      248.0
%                 1.0
dtype: float64

## Maximum gasUsed, gasUsedRP, Difference

In [21]:
sample.iloc[:,2:].apply(np.max)

Gas Used       3156235.00
Gas Used RP    2867038.00
Difference      289197.00
%                   25.38
dtype: float64

# 1Inch

In [22]:
contract = '1inch'
print('Total traced transactions:', len(results[contract]))

Total traced transactions: 7253


In [23]:
sample = get_sample(results[contract], new_columns, filter_columns)
sample.head()

,Block #,Transactions,Gas Used,Gas Used RP,Difference,%
0,0xbbf35e,0x62eedd32e8be32bfba8639085bda6826c96d19cf59fa...,100719,96336,4383,4.35
1,0xbbf35f,0x553f2c0568467b8fcb13ab905758138a7d4e5b73746b...,122707,116349,6358,5.18
2,0xbbf366,0xd6c7f4a1a1824223dc4a121741ceb5d6f8cc32b42b38...,113347,109084,4263,3.76
3,0xbbf366,0xd6c7f4a1a1824223dc4a121741ceb5d6f8cc32b42b38...,113347,109084,4263,3.76
4,0xbbf368,0x6661521ef9587895f06738a52bc243afa2291956252d...,112957,108872,4085,3.62


## Average gasUsed, gasUsedRP, Difference

In [24]:
sample.iloc[:,2:].apply(np.mean)

Gas Used       178508.723838
Gas Used RP    166468.228181
Difference      12040.495657
%                   5.708132
dtype: float64

## Minimum gasUsed, gasUsedRP, Difference

In [25]:
sample.iloc[:,2:].apply(np.min)

Gas Used       544.0
Gas Used RP    316.0
Difference     215.0
%                0.4
dtype: float64

## Maximum gasUsed, gasUsedRP, Difference

In [26]:
sample.iloc[:,2:].apply(np.max)

Gas Used       2430076.00
Gas Used RP    2406331.00
Difference      207364.00
%                   41.91
dtype: float64

# Metamask Swap

In [27]:
contract = 'mm_swap'
print('Total traced transactions:', len(results[contract]))

Total traced transactions: 13652


In [28]:
sample = get_sample(results[contract], new_columns, filter_columns)
sample.head()

,Block #,Transactions,Gas Used,Gas Used RP,Difference,%
0,0xbbf360,0x77727e8b2d4c08127db3dc41dc3f7895a7d9e6a375ef...,227112,215430,11682,5.14
1,0xbbf360,0xe8f077fb25e6b6f4aa5efaa515a5938f5c7ee3fed023...,191344,178657,12687,6.63
2,0xbbf360,0xe62f2eee5a4f2e074bd2649a43e2638bcbec610d129f...,175521,166693,8828,5.03
3,0xbbf360,0x1287cfde3241d8f169b45631699174d6f8a867fb15d5...,200870,188680,12190,6.07
4,0xbbf360,0xf96e8faf205829bc84be811279ddfc78aada61a8d446...,183736,162698,21038,11.45


## Average gasUsed, gasUsedRP, Difference

In [29]:
sample.iloc[:,2:].apply(np.mean)

Gas Used       210745.842953
Gas Used RP    198401.027835
Difference      12344.815119
%                   5.691159
dtype: float64

## Minimum gasUsed, gasUsedRP, Difference

In [30]:
sample.iloc[:,2:].apply(np.min)

Gas Used       563.0
Gas Used RP    320.0
Difference      72.0
%                0.1
dtype: float64

## Maximum gasUsed, gasUsedRP, Difference

In [31]:
sample.iloc[:,2:].apply(np.max)

Gas Used       2433208.00
Gas Used RP    2407720.00
Difference      254552.00
%                   43.16
dtype: float64

# Uniswap

In [32]:
contract = 'uniswap'
print('Total traced transactions:', len(results[contract]))

Total traced transactions: 77600


In [33]:
sample = get_sample(results[contract], new_columns, filter_columns)
sample.head()

,Block #,Transactions,Gas Used,Gas Used RP,Difference,%
0,0xbbcd65,0x5e6d5bf3c26e57f7660a1d8bcb3105fec8f3d6b6ce90...,106777,100023,6754,6.33
1,0xbbcd65,0xef6102ed1c3a4841aaae5d6fbb0e5cf139584a0776be...,83381,76505,6876,8.25
2,0xbbcd65,0x8a7eec8c1e392d0f2d8178731061024bff3f64f4c0c9...,121450,114766,6684,5.50
3,0xbbcd65,0xc5ad659bbb28dc475f7cae798ae752c5cc07f5608aaf...,65866,60376,5490,8.34
4,0xbbcd65,0x2c015811f7241d46e583b449850af1d9faed32c036a1...,118690,112303,6387,5.38


## Average gasUsed, gasUsedRP, Difference

In [34]:
sample.iloc[:,2:].apply(np.mean)

Gas Used       122717.535838
Gas Used RP    114881.337745
Difference       7836.198093
%                   9.391907
dtype: float64

## Minimum gasUsed, gasUsedRP, Difference

In [35]:
sample.iloc[:,2:].apply(np.min)

Gas Used       55.00
Gas Used RP    43.00
Difference     12.00
%               0.03
dtype: float64

## Maximum gasUsed, gasUsedRP, Difference

In [36]:
sample.iloc[:,2:].apply(np.max)

Gas Used       3549162.00
Gas Used RP    3513289.00
Difference       60638.00
%                   44.92
dtype: float64

# USDT

In [37]:
contract = 'usdt'
print('Total traced transactions:', len(results[contract]))

Total traced transactions: 183925


In [38]:
sample = get_sample(results[contract], new_columns, filter_columns)
sample.head()

,Block #,Transactions,Gas Used,Gas Used RP,Difference,%
0,0xbbf35e,0xfb9ace8a1556803fcdbebaa001f382f5ce90769103c5...,41601,40757,844,2.03
1,0xbbf35e,0x8e93fc8f381571d6b7793c12ad424791015142195f26...,41601,40757,844,2.03
2,0xbbf35e,0xf9f6fd000fbcd994f8f686f5b6ec7ed24220b056149b...,41601,40757,844,2.03
3,0xbbf35e,0x8d43e3236858466e64efbf0bf6c279f8f2b3e9f91d5b...,24501,23657,844,3.44
4,0xbbf35e,0x0cb835eecfb45125e45daf3daad1931d0ed850534301...,24501,23657,844,3.44


## Average gasUsed, gasUsedRP, Difference

In [39]:
sample.iloc[:,2:].apply(np.mean)

Gas Used       33170.483061
Gas Used RP    32213.608085
Difference       956.874976
%                  2.877546
dtype: float64

## Minimum gasUsed, gasUsedRP, Difference

In [40]:
sample.iloc[:,2:].apply(np.min)

Gas Used       21.00
Gas Used RP    15.00
Difference      6.00
%               0.02
dtype: float64

## Maximum gasUsed, gasUsedRP, Difference

In [41]:
sample.iloc[:,2:].apply(np.max)

Gas Used       5616221.00
Gas Used RP    5579274.00
Difference       89827.00
%                   43.16
dtype: float64

# USDC

In [42]:
contract = 'usdc'
print('Total traced transactions:', len(results[contract]))

Total traced transactions: 67406


In [43]:
sample = get_sample(results[contract], new_columns, filter_columns)
sample.head()

,Block #,Transactions,Gas Used,Gas Used RP,Difference,%
0,0xbbf35e,0xbedbafbae90513e0a2d21a82f0c8f31fefd18a4b382f...,26873,26205,668,2.49
1,0xbbf35e,0xf3cb9b4641f5699af6002195b165fe11945dd2c59055...,43973,43305,668,1.52
2,0xbbf35e,0x3151d17ec4d4ce3572a3fd23b20402e2ffd488485cea...,26873,26205,668,2.49
3,0xbbf35e,0x1d59a26f01981e8d708fd3382e784178b40bdc6a0450...,43973,43305,668,1.52
4,0xbbf35e,0x568a8658d0f533592fc0365fa6ea9c826268e12400c8...,26873,26205,668,2.49


## Average gasUsed, gasusedRP, Difference

In [44]:
sample.iloc[:,2:].apply(np.mean)

Gas Used       39438.348070
Gas Used RP    38634.609189
Difference       803.738881
%                  1.917254
dtype: float64

## Minimum gasUsed, gasUsedRP, Difference

In [45]:
sample.iloc[:,2:].apply(np.min)

Gas Used       55.00
Gas Used RP    43.00
Difference     12.00
%               0.33
dtype: float64

## Maximum gasUsed, gasUsedRP, Difference

In [46]:
sample.iloc[:,2:].apply(np.max)

Gas Used       2975184.00
Gas Used RP    2949957.00
Difference       83365.00
%                   43.16
dtype: float64

##### 